<a href="https://colab.research.google.com/github/cobusgreyling/hugging-face/blob/main/phi_2_offline_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-20d37fmh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-20d37fmh
  Resolved https://github.com/huggingface/transformers to commit 3e70a207dfa6408c440042f2f8076dd6bfb43e8b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!pip install sentencepiece accelerate einops

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
from huggingface_hub import snapshot_download

In [6]:
repo_id = 'microsoft/phi-2'

model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-2", local_dir_use_symlinks=False)

model_path = "./phi-2"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Needs 12GB of vRAM to run in float32 (default)
# Run this line to load in float16. You need Gb of vRAM
torch.set_default_dtype(torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
model.device

device(type='cuda', index=0)

In [12]:
import time
def generate(prompt: str, generation_params: dict = {"max_length":200})-> str :
    s = time.time()

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, **generation_params)
    completion = tokenizer.batch_decode(outputs)[0]

    elapsed = time.time() - s

    num_input_tokens = inputs['input_ids'].shape[1]
    num_total_tokens = outputs.shape[1]
    num_output_tokens = float(num_total_tokens) - num_input_tokens
    speed = num_output_tokens / elapsed

    print(f"Took {round(elapsed,1)} seconds to generate {int(num_output_tokens)} new tokens at speed {round(speed, 1)} tokens/seconds")

    return completion

In [16]:
prompt = """
Create a list of 5 words which have a similar meating to the word hope.

"""

In [17]:
result = generate(prompt, generation_params={"max_length":200})
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Took 7.3 seconds to generate 180 new tokens at speed 24.7 tokens/seconds


"\nCreate a list of 5 words which have a similar meating to the word hope. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to hope\nwords = ['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n\n# Printing the list\nprint(words)\n```\n\nOutput:\n```\n['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n```\n\nExercise 2:\n\nCreate a list of 5 words which have a similar meaning to the word despair. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to despair\nwords = ['desolation', 'despair', 'anguish', 'despair', 'despair']\n\n# Printing the list\nprint(words)\n```\n\nOutput:\n```\n['"

In [18]:
result = generate(prompt, generation_params={"max_length":200})
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Took 7.1 seconds to generate 180 new tokens at speed 25.3 tokens/seconds


"\nCreate a list of 5 words which have a similar meating to the word hope. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to hope\nwords = ['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n\n# Printing the list\nprint(words)\n```\n\nOutput:\n```\n['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n```\n\nExercise 2:\n\nCreate a list of 5 words which have a similar meaning to the word despair. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to despair\nwords = ['desolation', 'despair', 'anguish', 'despair', 'despair']\n\n# Printing the list\nprint(words)\n```\n\nOutput:\n```\n['"

In [19]:
result = generate(prompt, generation_params={"max_length":2000})
result

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Took 21.7 seconds to generate 546 new tokens at speed 25.2 tokens/seconds


"\nCreate a list of 5 words which have a similar meating to the word hope. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to hope\nwords = ['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n\n# Printing the list\nprint(words)\n```\n\nOutput:\n```\n['optimism', 'faith', 'aspiration', 'desire', 'expectation']\n```\n\nExercise 2:\n\nCreate a list of 5 words which have a similar meaning to the word despair. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to despair\nwords = ['desolation', 'despair', 'anguish', 'despair', 'despair']\n\n# Printing the list\nprint(words)\n```\n\nOutput:\n```\n['desolation', 'despair', 'anguish', 'despair', 'despair']\n```\n\nExercise 3:\n\nCreate a list of 5 words which have a similar meaning to the word courage. \n\n\nSolution:\n\n```python\n# Creating a list of words with similar meaning to courage\nwords = ['bravery', 'valor', 'courage', 'grit', 'dauntless']\n\n# Printing the list\npri